In [2]:
from transformers import pipeline

pipe = pipeline("token-classification", model="gyr66/bert-base-chinese-finetuned-ner", device=1)

In [3]:
def transform_output(entities, _id):
    result = []
    i = 0
    while i < len(entities):
        entity = entities[i]
        # Remove the 'B-' or 'I-' prefix from the category
        category = entity['entity'][2:]
        word = entity['word']
        start = entity['start']
        end = entity['end'] - 1  # Subtract 1 from the end position
        # Check if there are 'I-' entities following this 'B-' entity
        while i + 1 < len(entities) and entities[i + 1]['entity'].startswith('I-'):
            i += 1
            word += entities[i]['word']
            end = entities[i]['end'] - 1  # Subtract 1 from the end position
        result.append({
            'ID': _id,
            'Category': category,
            'Pos_b': start, 
            'Pos_e': end,
            'Privacy': word
        })
        i += 1
    return result

In [6]:
import pandas as pd
import os

In [7]:
text_list = []
id_list = []
res = []
for filename in sorted(os.listdir('data/original/test/text'), key=lambda x: int(x.split('.')[0])):
    with open(os.path.join('data/original/test/text', filename), 'r') as f:
        text = f.read()
    id_list.append(filename.split('.')[0])
    text_list.append(text)
output_list = pipe(text_list, batch_size=64)
for _id, output in zip(id_list, output_list):
    output = transform_output(output, _id)
    res.extend(output)
df = pd.DataFrame(res)
df.to_csv('data/original/test/predict.csv', encoding='utf-8', index=False)

  0%|          | 0/3956 [00:00<?, ?it/s]